Let us have a function $y_i = f(x_i)$, defined on grid template $\Omega_n$ for $i = \overline{0,n}$; moreover, $x_i \in [a, b]$. We are tasked to complete the function by constructing differential cubic spline with defect q = 1, that is, construct function $S_3(x) \in C_2[a, b]$ using chunk-global method. We also suppose that $y_i = f(x_i)$ is smooth enough itself.

Let us approximate each chunk using the following polynomial defined on $x \in [x_i, x_{i + 1}]$

$$
S_{3,i}(x) = a_{0,i} + a_{1,i}(x - x_i) + a_{2,i}(x - x_i)^2 + a_{3,i}(x - x_i)^3
$$

with this polynomial being the i-th link of the spline. Now we choose the following compatibility conditions:

Functional:

$$
\delta S_{3,i}(x)|_{x=x_i} = S_{3,i}(x_i) - f(x_i) = 0
$$
$$
\delta S_{3,i}(x)|_{x=x_{i + 1}} = S_{3,i}(x_{i + 1}) - f(x_{i + 1}) = 0
$$

Differential:

$$
\delta S'_{3,i}(x)|_{x=x_i} = S'_{3,i}(x_i) - f'(x_i) = 0
$$
$$
\delta S'_{3,i}(x)|_{x=x_{i + 1}} = S'_{3,i}(x_{i + 1}) - f'(x_{i + 1}) = 0
$$

$$
\delta S''_{3,i}(x)|_{x=x_i} = S''_{3,i}(x_i) - f''(x_i) = 0
$$
$$
\delta S''_{3,i}(x)|_{x=x_{i + 1}} = S''_{3,i}(x_{i + 1}) - f''(x_{i + 1}) = 0
$$

For maintaining the continuity of the spline function and its first and second derivatives we also need to apply the following stitching conditions for $i = \overline{0,n-1}$:

$$
S_{3,i-1}(x)|_{x=x_i} = S_{3,i}(x)|_{x=x_i}
$$
$$
S'_{3,i-1}(x)|_{x=x_i} = S'_{3,i}(x)|_{x=x_i}
$$
$$
S''_{3,i-1}(x)|_{x=x_i} = S''_{3,i}(x)|_{x=x_i}
$$

The functional compatibility conditions guarantee that the approximating function will pass through grid points of the approximated function.

The differential compatibility conditions guarantee that the approxiamting function derivatives will be the same as approximated function corresponding derivatives at grid points, at least with some precision.

The functional stitching conditions guarantee the continuity of the spline function on $x \in [a,b]$ when it is constructed from spline links.

The differential stitching conditions guarantee the continuity of the spline function derivatives on $x \in [a,b]$ when the spline is constructed from spline links.

Now we will substitute the polynomial for $S_{3,i}(x)$ into the functional and 2-nd derivative compatibility
conditions and rewrite the acquired system of algebraic equations in matrix form:

$$
\begin{pmatrix}
1 & 0 & 0 & 0 \\
1 & x_{i+1} - x_i & (x_{i+1} - x_i)^2 & (x_{i+1} - x_i)^3 \\
0 & 0 & 2 & 0 \\
0 & 0 & 2 & 6(x_{i+1} - x_i)
\end{pmatrix}
\times
\begin{bmatrix}
a_{0,i} \\
a_{1,i} \\
a_{2,i} \\
a_{3,i}
\end{bmatrix}
=
\begin{bmatrix}
f_i \\
f_{i+1} \\
m_i \\
m_{i+1}
\end{bmatrix},
$$

where $f_i = f(x_i)$ and $m_i = f''(x_i),\ i=\overline{0,n}$. Solving this system, which can be done by left
multiplying both sides by

$$
\begin{pmatrix}
1 & 0 & 0 & 0 \\
-\frac{1}{x_{i+1} - x_i} & \frac{1}{x_{i+1} - x_i} & \frac{2}{3}(x_{i+1} - x_i) & -\frac{x_{i+1} - x_i}{6} \\
0 & 0 & \frac{1}{2} & 0 \\
0 & 0 & -\frac{1}{6(x_{i+1} - x_i)} & \frac{1}{6(x_{i+1} - x_i)}
\end{pmatrix},
$$

and deriving the formulas for $a_{t,i},\ t=\overline{0,3}$, we get

$$
S_{3,i}(x) = f_i + \bigg(\frac{1}{h_{i+1}} \Delta f_i - \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}\Delta m_i\bigg)
(x - x_i) + \frac{m_i}{2}(x - x_i)^2 + \frac{1}{6h_{i+1}}\Delta m_i(x - x_i)^3,\ i = \overline{1,n-1},
$$

where $h_{i+1} = x_{i+1} - x_i,\ \Delta f_i = f_{i+1} - f_i,\ \Delta m_i = m_{i+1} - m_i$. Now by substituting the
formula for $S_{3,i}(x)$ into the first-order differential stitching condition, we get:

$$
\frac{1}{h_i}\Delta f_{i-1} - \frac{h_i}{2}m_{i-1} - \frac{h_i}{6}\Delta m_{i-1} + m_{i-1}(x_i - x_{i-1})
+ \frac{1}{2h_i}\Delta m_{i-1}(x_i - x_{i-1})^2 = \frac{1}{h_{i+1}}\Delta f_i - \frac{h_{i+1}}{2}m_i
- \frac{h_{i+1}}{6}\Delta m_i
$$

$$
-\frac{h_i}{2}m_{i-1} - \frac{h_i}{6}(m_i - m_{i-1}) + m_{i-1} h_i + \frac{h_i}{2}(m_i - m_{i-1}) + \frac{h_{i+1}}{2}
m_i + \frac{h_{i+1}}{6}(m_{i+1} - m_i) = \frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i}
$$

$$
\bigg(- \frac{h_i}{2}m_{i-1} + \frac{h_i}{6}m_{i-1} + m_{i-1} h_i - \frac{h_i}{2}m_{i-1}\bigg)
+ \bigg(- \frac{h_i}{6}m_i + \frac{h_i}{2}m_i + \frac{h_{i+1}}{2}m_i - \frac{h_{i+1}}{6}m_i\bigg)
+ \frac{h_{i+1}}{6}m_{i+1}
= \frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i}
$$

$$
\frac{h_i}{6}m_{i-1} + \frac{h_i + h_{i+1}}{3}m_i + \frac{h_{i+1}}{6}m_{i+1}
= \frac{\Delta f_i}{h_{i+1}} - \frac{\Delta f_{i-1}}{h_i},\ i = \overline{1,n-1}
$$

The last equation clearly shows the dependence between unknown parameters $m_{i-1},\ m_i,\ m_{i+1}$ which are
values of the second derivatives of $y_i = f(x_i)$ in corresponding points on the grid template on the
differences of function values in those grid points ($\Delta f_{i-1},\ \Delta f_i$).

Now we can create a system of equations with entry for each $i = \overline{0,n}$:

$$
\begin{pmatrix}
? & ? & ? & 0 & \cdots & 0 & 0 & 0 & 0 \\
\frac{h_1}{6} & \frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & \frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & \cdots & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-3} + h_{n-2}}{3} & \frac{h_{n-2}}{6} & 0 & 0 \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_n}{3} & \frac{h_n}{6} \\
0 & 0 & 0 & 0 & \cdots & 0 & ? & ? & ?
\end{pmatrix}
\times
\begin{bmatrix}
m_0 \\
m_1 \\
m_2 \\
m_3 \\
\vdots \\
m_{n-3} \\
m_{n-2} \\
m_{n-1} \\
m_n
\end{bmatrix}
=
\begin{bmatrix}
? \\
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
\frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
\vdots \\
\frac{\Delta f_{n-3}}{h_{n-2}} - \frac{\Delta f_{n-4}}{h_{n-2}} \\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-1}} \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}} \\
?
\end{bmatrix}
$$

As you can see the system is not closed because we have only $n+1$ unknowns in $n-1$ conditions. That is the
reason question marks stand in the top and bottom of the matrix and resulting vector. There are several ways to
overcome this difficulty which arises from the fact that we must manually set the stitching conditions on the ends
of $[a,b]$. The simplest way is to set $m_0 = 0,\ m_n = 0$, this way is called _natural_. In this case the system becomes

$$
\begin{pmatrix}
1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 \\
\frac{h_1}{6} & \frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & \frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & \cdots & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-3} + h_{n-2}}{3} & \frac{h_{n-2}}{6} & 0 & 0 \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_n}{3} & \frac{h_n}{6} \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 1
\end{pmatrix}
\times
\begin{bmatrix}
0 \\
m_1 \\
m_2 \\
m_3 \\
\vdots \\
m_{n-3} \\
m_{n-2} \\
m_{n-1} \\
0
\end{bmatrix}
=
\begin{bmatrix}
0 \\
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
\frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
\vdots \\
\frac{\Delta f_{n-3}}{h_{n-2}} - \frac{\Delta f_{n-4}}{h_{n-2}} \\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-1}} \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}} \\
0
\end{bmatrix}
$$

In case of constant grid, $m_0$ and $m_n$ can be calculated directly using the second-order derivative
approximation formulas:

$$
m_0 = \frac{1}{h^2}(2f_0 - 5f_1 + 4f_2 - f_3)
$$
$$
m_1 = \frac{1}{h^2}(-f_{n-3} + 4f_{n-2} - 5f_{n-1} + 2f_n)
$$

giving the system

$$
\begin{pmatrix}
1 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 0 \\
\frac{h_1}{6} & \frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & \frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & \cdots & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-3} + h_{n-2}}{3} & \frac{h_{n-2}}{6} & 0 & 0 \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_n}{3} & \frac{h_n}{6} \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & 0 & 1
\end{pmatrix}
\times
\begin{bmatrix}
m_0 \\
m_1 \\
m_2 \\
m_3 \\
\vdots \\
m_{n-3} \\
m_{n-2} \\
m_{n-1} \\
m_n
\end{bmatrix}
=
\begin{bmatrix}
\frac{1}{h^2}(-f_{n-3} + 4f_{n-2} - 5f_{n-1} + 2f_n) \\
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
\frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
\vdots \\
\frac{\Delta f_{n-3}}{h_{n-2}} - \frac{\Delta f_{n-4}}{h_{n-2}} \\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-1}} \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}} \\
\frac{1}{h^2}(-f_{n-3} + 4f_{n-2} - 5f_{n-1} + 2f_n)
\end{bmatrix},
$$

which can even be converted to tridiagonal, but the process is long and tedious, so it is not presented here. In
the case when the grid is variable one can apply conditions of equality of the third derivative for two first and
two last spline links

$$
\frac{m_k - m_{k-1}}{h_k} = \frac{m_{k+1} - m_k}{h_{k+1}},\ k=1,n-1
$$

This gives us two additional conditions we need

$$
\frac{1}{h_1}m_0 - \bigg(\frac{1}{h_1} + \frac{1}{h_2}\bigg)m_1 + \frac{1}{h_2}m_2 = 0
$$
$$
\frac{1}{h_{n-1}}m_{n-2} - \bigg(\frac{1}{h_{n-1}} + \frac{1}{h_n}\bigg)m_{n-1} + \frac{1}{h_n}m_n = 0
$$

Including those conditions into the system gives us

$$
\begin{pmatrix}
\frac{1}{h_1} & \frac{1}{h_1} + \frac{1}{h_2} & \frac{1}{h_2} & 0 & \cdots & 0 & 0 & 0 & 0 \\
\frac{h_1}{6} & \frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & \frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & \cdots & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-3} + h_{n-2}}{3} & \frac{h_{n-2}}{6} & 0 & 0 \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_n}{3} & \frac{h_n}{6} \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{1}{h_{n-1}} & \frac{1}{h_{n-1}} + \frac{1}{h_n} & \frac{1}{h_n}
\end{pmatrix}
\times
\begin{bmatrix}
m_0 \\
m_1 \\
m_2 \\
m_3 \\
\vdots \\
m_{n-3} \\
m_{n-2} \\
m_{n-1} \\
m_n
\end{bmatrix}
=
\begin{bmatrix}
0 \\
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
\frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
\vdots \\
\frac{\Delta f_{n-3}}{h_{n-2}} - \frac{\Delta f_{n-4}}{h_{n-2}} \\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-1}} \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}} \\
0
\end{bmatrix}
$$

The system above can be converted to tridiagonal by left multiplying both sides by

$$
\begin{pmatrix}
1 & -6\frac{1}{h_2^2} & 0 & \cdots & 0 & 0 & 0 \\
0 & 1 & 0 & \cdots & 0 & 0 & 0 \\
0 & 0 & 1 & \cdots & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & \cdots & 1 & 0 & 0 \\
0 & 0 & 0 & \cdots & 0 & 1 & 0 \\
0 & 0 & 0 & \cdots & 0 & -6\frac{1}{h_{n-1}^2} & 1
\end{pmatrix}
$$

giving us a final result of

$$
\begin{pmatrix}
\frac{1}{h_1} - \frac{h_1}{h_2^2} & \frac{1}{h_1} + \frac{1}{h_2} - 2\frac{h_1 + h_2}{h_2^2} & 0 & 0 & \cdots & 0 & 0 & 0 & 0 \\
\frac{h_1}{6} & \frac{h_1 + h_2}{3} & \frac{h_2}{6} & 0 & \cdots & 0 & 0 & 0 & 0 \\
0 & \frac{h_2}{6} & \frac{h_2 + h_3}{3} & \frac{h_3}{6} & \cdots & 0 & 0 & 0 & 0 \\
0 & 0 & \frac{h_3}{6} & \frac{h_3 + h_4}{3} & \cdots & 0 & 0 & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-3} + h_{n-2}}{3} & \frac{h_{n-2}}{6} & 0 & 0 \\
0 & 0 & 0 & 0 & \cdots & \frac{h_{n-2}}{6} & \frac{h_{n-2} + h_{n-1}}{3} & \frac{h_{n-1}}{6} & 0 \\
0 & 0 & 0 & 0 & \cdots & 0 & \frac{h_{n-1}}{6} & \frac{h_{n-1} + h_n}{3} & \frac{h_n}{6} \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & \frac{1}{h_{n-1}} + \frac{1}{h_n} - 2\frac{h_{n-1} + h_n}{h_{n-1}^2} & \frac{1}{h_n} - \frac{h_n}{h_{n-1}^2}
\end{pmatrix}
\times
\begin{bmatrix}
m_0 \\
m_1 \\
m_2 \\
m_3 \\
\vdots \\
m_{n-3} \\
m_{n-2} \\
m_{n-1} \\
m_n
\end{bmatrix}
=
\begin{bmatrix}
6\bigg(\frac{\Delta f_0}{h_1 h_2^2} - \frac{\Delta f_1}{h_2^3}\bigg) \\
\frac{\Delta f_1}{h_2} - \frac{\Delta f_0}{h_1} \\
\frac{\Delta f_2}{h_3} - \frac{\Delta f_1}{h_2} \\
\frac{\Delta f_3}{h_4} - \frac{\Delta f_2}{h_3} \\
\vdots \\
\frac{\Delta f_{n-3}}{h_{n-2}} - \frac{\Delta f_{n-4}}{h_{n-2}} \\
\frac{\Delta f_{n-2}}{h_{n-1}} - \frac{\Delta f_{n-3}}{h_{n-1}} \\
\frac{\Delta f_{n-1}}{h_n} - \frac{\Delta f_{n-2}}{h_{n-1}} \\
6\bigg(\frac{\Delta f_{n-2}}{h_{n-1}^3} - \frac{\Delta f_{n-1}}{h_n h_{n-1}^2}\bigg)
\end{bmatrix}
$$

Let us have the system of algebraic equations defined by the following series:

$$
\alpha_i x_{i-1} - \beta_i x_i + \gamma x_{i+1} = \delta_i,\ \alpha_i = \gamma_n = 0,\ i = \overline{1,n}
$$

Rewriting these series in extended matrix form we get

$$
A_1 = \begin{pmatrix}
-\beta_1 & \gamma_1 & 0 & 0 & \cdots & 0 & 0 & \delta_1 \\
\alpha_2 & -\beta_2 & \gamma_2 & 0 & \cdots & 0 & 0 & \delta_2 \\
0 & \alpha_3 & -\beta_3 & \gamma_3 & \cdots & 0 & 0 & \delta_3 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & \alpha_n & -\beta_n & \delta_n
\end{pmatrix}
$$

Let us find the solution of the system using Gauss method. First of all, let us convert the matrix to the upper
diagonal form using direct pass of the Gauss method:

$$
\hat{A}_1 = \begin{pmatrix}
1 & -P_1 & 0 & 0 & \cdots & 0 & 0 & Q_1 \\
0 & 1 & -P_2 & 0 & \cdots & 0 & 0 & Q_2 \\
0 & 0 & 1 & -P_3 & \cdots & 0 & 0 & Q_3 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & 0 & 1 & Q_n
\end{pmatrix}
$$

Now we can derive the recurrent formula for $x_i,\ i=\overline{1,n}$. Note that $x_i - x_{i+1}P_i = Q_i$. Then

$$
x_i = P_i x_{i+1} + Q_i
$$

For $i-1$

$$
x_{i-1} = P_{i-1} x_i + Q_{i-1}
$$

Let us substitute this expression into the original formula:

$$
\alpha_i (P_{i-1} x_i + Q_{i-1}) - \beta_i x_i + \gamma x_{i+1} = \delta_i
$$

$$
\alpha_i P_{i-1} x_i + \alpha_i Q_{i-1} - \beta_i x_i + \gamma x_{i+1} = \delta_i
$$

$$
x_i (\beta_i - \alpha_i P_{i-1}) = \alpha_i Q_{i-1} + \gamma x_{i+1} - \delta_i
$$

$$
x_i = \frac{\gamma}{\beta_i-\alpha_i P_{i-1}}x_{i+1} + \frac{\alpha_i Q_{i-1}-\delta_i}{\beta_i-\alpha_i P_{i-1}}
$$

From that we also get a recurrent formula for $P_i$ and $Q_i$:

$$
P_i = \frac{\gamma_i}{\beta_i-\alpha_i P_{i-1}},\ Q_i = \frac{\alpha_i Q_{i-1}-\delta_i}{\beta_i-\alpha_i P_{i-1}}
,\ i=\overline{1,n-1}
$$

Considering that $P_0 = 0,\ Q_0 = 0$, because this way

$$
P_1 = \frac{\gamma_i}{\beta_i},\ Q_1 = -\frac{\delta_i}{\beta_i}
$$

To start the reverse process we need to calculate $x_n$ by using two equations:

$$
x_{n-1} = P_{n-1}x_n + Q_{n-1}
$$
$$
\alpha_n x_{n-1} - \beta_n x_n + \gamma_n \cdot 0 = \delta_n
$$

By subtituting $x_{n-1}$:

$$
\alpha_n (P_{n-1}x_n + Q_{n-1}) - \beta_n x_n + \gamma_n \cdot 0 = \delta_n
$$

$$
x_n = \frac{\alpha_n Q_{n-1} - \delta_n}{\beta_n - \alpha_n P_{n-1}}
$$

**Summary**

Direct pass: calculate $P_i,\ Q_i,\ i=\overline{1,n-1}$ using

$$
P_1 = \frac{\gamma_i}{\beta_i},\ Q_1 = -\frac{\delta_i}{\beta_i}
$$

$$
P_i = \frac{\gamma_i}{\beta_i-\alpha_i P_{i-1}},\ Q_i = \frac{\alpha_i Q_{i-1}-\delta_i}{\beta_i-\alpha_i P_{i-1}}
$$

Reverse pass: calculate $x_i,\ i=\overline{n,1}$ using

$$
x_n = \frac{\alpha_n Q_{n-1} - \delta_n}{\beta_n - \alpha_n P_{n-1}}
$$

$$
x_i = P_i x_{i+1} + Q_i
$$

Here is the prototype of the algorithm in C++:

```cpp
const size_t SIZE = SOME_PREDEFINED_SIZE;
const double EPS = 1e-9;
size_t N, M, K;
double knots[SIZE], h[SIZE], func_values[SIZE], delta_f[SIZE], m[SIZE]; // Input data, apart from m
double matrix[SIZE][SIZE];
double x[SIZE], P[SIZE], Q[SIZE]; // For matrix solving
double S[SIZE][SIZE][4]; // Holds spline links parameters
double points[SIZE]; // Points of interpolation
size_t indices[SIZE];

inline int compare_doubles(double, double);
inline void bin_search_index(size_t, size_t, double *knots, double *points, size_t *indices);

inline void construct_matrix(size_t, double *, double *, double *, double [][SIZE], double *);
inline void solve_matrix(size_t, size_t, double *, double *, double *, double *, double [][SIZE], double *, double *, double *, double [][SIZE][4]);

int main(int argc, char **argv);
```

The two essential functions are `construct_matrix` which produces a matrix for the task as shown above and
`solve_matrix` that uses sweep method to solve the constructed system. `compare_doubles` and `bin_search_index`
are helper functions for finding the required spline link for the point of interpolation. The `main` function is
just standard C++ entrypoint.

Let us estimate the method error. Assuming that $f(x_i) \in C_4[a,b]$ it can be proven that

$$
\max_{[a,b]} |f^{(p)}(x) - S_3^{(p)}(x)| \leq M_4 h^{4-p},\ p = 0,1,2
$$

But what if $f(x_i) \notin C_4[a,b]$? Then why are we even bothering ourselves with constructing Cubic Spline
interpolation for a function that does not even have 4-th derivative? We should use a simpler method for better
results.

Now that we developed all the needed mathematical machinery, we should estimate the complexity of the algorithm of
cubic spline interpolation. The construction of the matrix is done in $O(n)$ as it's tridiagonal. The direct pass
of the algorithm takes $O(n)$ as does the reverse pass.

**Complexity of the spline construction algorithm**: $O(N)$, where N is the number of knots in grid template
$\Omega$.

**Complexity of the homework**: $O(M * (N + K))$, where N is the number of knots in grid template $\Omega$ and M
is the number of interpolation points' sets, K is the number of interpolation points in each set.